# ReGroup Demo

Welcome to ReGroup Demo notebook.

# Part I. Finding & correcting the symmetry of a distorted polytope

In [11]:
import ipywidgets as wdg
import numpy as np
from IPython.display import display, clear_output, Markdown
import polytope as ptp
from math import sqrt
from symmfinder import symmetry_finder, inclusive_closure
from grpcorr import multab_group_correction

## 1. Select polytope type:

In [12]:

pchoice = wdg.RadioButtons( options=["hypercube","hypercube dual polytope","simplex",
                                    "dodecahedron (3D)","icositetrachoron (4D)", "hexacosichoron (4D)"], 
                           description = "Polytope")
dchoice = wdg.IntSlider(min=1, max=7, disabled=False, value = 1, description = 'Dimension')

def dchoice_update(*args):
    off = False
    if pchoice.index == 3:
        dchoice.value=3 
        dchoice.disabled = True
    elif pchoice.index in [4,5]:
        dchoice.value=4
        dchoice.disabled = True
    else:
        dchoice.disabled = False

def print_ptope(x,y):
    print("You selected: ",x,", D = ", y)
        
pchoice.observe(dchoice_update,'index')
        
wdg.interactive(print_ptope,x=pchoice,y=dchoice)

interactive(children=(RadioButtons(description='Polytope', options=('hypercube', 'hypercube dual polytope', 's…

## 2. Enter polytope radius:

In [13]:
rchoice = wdg.BoundedFloatText(description = "Radius", step = .05, min = 0e0)
display(rchoice)

BoundedFloatText(value=0.0, description='Radius', step=0.05)

## 3. Random distortion:

In [14]:
distort = wdg.Checkbox(description = "Apply random distortion", value = True)
epschoice = wdg.BoundedFloatText(description = "Amplitude", step = .05, min = 0e0)

def epschoice_update(*args):
    epschoice.disabled = not distort.value
    if not distort.value:
        epschoice.value = 0e0
    
distort.observe(epschoice_update,"value")

def print_distort(d,eps):
    print("Random distortion: ",d,"; amplitude = ", eps)
    
wdg.interactive(print_distort, d=distort, eps = epschoice)

interactive(children=(Checkbox(value=True, description='Apply random distortion'), BoundedFloatText(value=0.0,…

## 4. Generate the points:

In [32]:
bcreate = wdg.Button(description = "Create polytope")
bclear = wdg.Button(description = "Clear points")
points_input = wdg.Textarea(layout=wdg.Layout(width='100%'))

def create_polytope(*args):
    i = pchoice.index
    dim = dchoice.value
    a = rchoice.value
    eps = epschoice.value
    if i==0:
        points = ptp.hypercube(2*a/sqrt(dim),dim)
    elif i==1:
        points = ptp.hypercube_dual(a,dim)
    elif i==2:
        points = ptp.simplex(a,dim)
    elif i==3:
        points = ptp.dodecahedron(a)
    elif i==4:
        points = ptp.icositetrachoron(a)
    elif i==5:
        points = ptp.hexacosichoron(a)
    else:
        pass
    if distort.value:
        ptp.random_distortion(points,eps)
    points_input.value = '\n'.join([('{:10.6f}'*len(p)).format(*p) for p in points])
    
def clear_points(*args):
    points_input.value = ""
    
bcreate.on_click(create_polytope)
bclear.on_click(clear_points)
display(wdg.HBox([bcreate,bclear]))
display(Markdown("## 5. Use generated points or (alternatively) provide them manually"))
display(points_input)

## 5. Use generated points or (alternatively) provide them manually

Textarea(value='', layout=Layout(width='100%'))

## 6. Select epsilon (tolerance) value for approximate symmetry operations

In [16]:
eps1choice = wdg.BoundedFloatText(description = "Tolerance", step = .05, min = 0e0)
display(eps1choice)

BoundedFloatText(value=0.0, description='Tolerance', step=0.05)

## 7. Find approximate symmetry operations
When points are created (step 5) and epsilon value is chosen, run the following steps manually:

In [19]:
points = [np.array([float(s) for s in l.split()]) for l in points_input.value.split('\n')]
dim = dchoice.value
eps = eps1choice.value
print(dim,eps)
P,G = symmetry_finder(points,dim,eps)

6 0.01
Dot products matrix:    0.004 sec CPU time
Point to point list:    0.002 sec CPU time
Pair to pair list:      2.237 sec CPU time
     46080 approximate symmetry operations found
Approximate symmetries search:    325.649 sec CPU time


## 8. Perform closure

In [8]:
Multab = inclusive_closure(P,G)

Inclusive closure & multiplication table build:
Multiplication table: 100.00% complete,    0.185 sec CPU time
       240 new elements found,    0.189 sec CPU time spent
Multiplication table: 100.00% complete,    1.617 sec CPU time
         0 new elements found,    1.617 sec CPU time spent
Total group size now:    384


## 9. Multiplication table based group correction

In [9]:
multab_group_correction(G,Multab,eps=1e-10)

simple_grp_correct: iteration    0 error      4.836861301248423395
simple_grp_correct: iteration    1 error      0.000604377624966258
simple_grp_correct: iteration    2 error      0.000000000019039891
Convergence is achieved!


In [18]:
G[1]

array([[ 1.00000000e+00, -9.78796924e-52,  7.52317027e-35,
         7.52317027e-35, -8.67362108e-18],
       [ 0.00000000e+00,  1.00000000e+00, -1.69985408e-17,
         1.69985408e-17,  3.76158514e-35],
       [ 0.00000000e+00,  1.69985408e-17,  1.00000000e+00,
        -3.56682708e-16,  8.67362108e-18],
       [ 8.67362108e-18,  1.12847554e-34, -8.67362108e-18,
        -8.67362108e-18,  1.00000000e+00],
       [ 0.00000000e+00, -1.69985408e-17,  7.78822944e-17,
         1.00000000e+00,  8.67362108e-18]])



# Part II. Application to molecular symmetry 

In [60]:
import nglview as ngl
from ase import Atom, Atoms
import ipywidgets as wdg
import numpy as np
from IPython.display import display, clear_output, Markdown
import polytope as ptp
from symmfinder import symmetry_finder, inclusive_closure
from grpcorr import multab_group_correction

## 1. Read molecule from xyz file
The xyz file format is simple:

*A1 x1 y1 z1*

*A2 x2 y2 z2*

............

*An xn yn zn*

First two lines might also contain the number of atoms and some comment, however, it is not necessary. The limited number of ready-to-go examples include "examples/c20.xyz", "examples/ch4.xyz", "examples/c2h6.xyz" and "examples/sf6.xyz".

In [54]:
fname = wdg.Text(description = 'xyz file:', value = 'examples/c20.xyz')
bopen = wdg.Button(description = 'Open')
mol_input = wdg.Textarea(layout=wdg.Layout(width='70%'))

def open_xyz_file(*args):
    f = open(fname.value)
    mol_input.value = ''.join([l for l in f])
    f.close()

bopen.on_click(open_xyz_file)
display(wdg.HBox([fname,bopen]),Markdown('##  2. Alternatively, copy & paste it here'),mol_input)

##  2. Alternatively, copy & paste it here

Textarea(value='', layout=Layout(width='70%'))

## 3. Apply random distortion

In [56]:
distort = wdg.Checkbox(description = "Apply random distortion", value = True)
epschoice = wdg.BoundedFloatText(description = "Amplitude", step = .05, min = 0e0)

def epschoice_update(*args):
    epschoice.disabled = not distort.value
    if not distort.value:
        epschoice.value = 0e0
    
distort.observe(epschoice_update,"value")

def print_distort(d,eps):
    print("Random distortion: ",d,"; amplitude = ", eps)
    
wdg.interactive(print_distort, d=distort, eps = epschoice)

interactive(children=(Checkbox(value=True, description='Apply random distortion'), BoundedFloatText(value=0.0,…

## 4. Read & visualize initial molecule
Manually run the following code

In [70]:
lines=[line.split() for line in mol_input.value.split('\n')]
mol = {'atoms':[],'coord':[]}
for l in lines:
    if len(l)==4:
        mol['atoms'].append(l[0])
        mol['coord'].append(np.array([float(l[i]) for i in [1,2,3]]))
        
if distort.value:
    ptp.random_distortion(mol['coord'],epschoice.value)

ase_mol = Atoms([Atom(mol['atoms'][i], tuple(mol['coord'][i])) for i in range(len(mol['atoms']))])
mol_view = ngl.show_ase(ase_mol)
display(mol_view)

NGLWidget()

## 5. Select epsilon (tolerance) value for approximate symmetry operations

In [63]:
eps1choice = wdg.BoundedFloatText(description = "Tolerance", step = .05, min = 0e0)
display(eps1choice)

BoundedFloatText(value=0.0, description='Tolerance', step=0.05)

## 6. Find approximate symmetry group

In [72]:
eps = eps1choice.value
P,G = symmetry_finder(mol['coord'],3,eps)
Multab = inclusive_closure(P,G)

Dot products matrix:    0.001 sec CPU time
Point to point list:    0.000 sec CPU time
Pair to pair list:      0.000 sec CPU time
         5 approximate symmetry operations found
Approximate symmetries search:      0.020 sec CPU time
Inclusive closure & multiplication table build:
Multiplication table: 100.00% complete,    0.001 sec CPU time
         6 new elements found,    0.002 sec CPU time spent
Multiplication table: 100.00% complete,    0.001 sec CPU time
         1 new elements found,    0.001 sec CPU time spent
Multiplication table: 100.00% complete,    0.000 sec CPU time
         0 new elements found,    0.001 sec CPU time spent
Total group size now:     12


## 7. Apply multiplication table based correction and symmetrize molecule

In [73]:
def symmetrize_mol(mol,G,P):
    mol1 = [np.array([0e0,0e0,0e0]) for m in mol['coord']]
    for i in range(len(G)):
        for j in range(len(mol['coord'])):
            k = P[i][j]
            mol1[k] = mol1[k] + G[i].dot(mol['coord'][j])
    for i in range(len(mol1)):
        mol1[i] /= len(G)
    mol['coord'] = mol1
    return mol

def print_xyz(mol):
    print(len(mol['atoms']))
    print()
    for i in range(len(mol['atoms'])):
        print('{:2s} {:10.5f} {:10.5f} {:10.5f}'.format(mol['atoms'][i],*mol['coord'][i]))

multab_group_correction(G,Multab,eps=1e-12)
mol = symmetrize_mol(mol,G,P)
display(Markdown('### Symmetrized molecule in xyz format:'))
print_xyz(mol)    

simple_grp_correct: iteration    0 error      0.868622102083505654
simple_grp_correct: iteration    1 error      0.001607373970717430
simple_grp_correct: iteration    2 error      0.000000010959657232
simple_grp_correct: iteration    3 error      0.000000000000005543
Convergence is achieved!


### Symmetrized molecule in xyz format:

8

C     0.02215    0.02273    0.66399
C    -0.02215   -0.02273   -0.66399
H     0.53746    0.76618    1.13106
H    -0.83934    0.10941    1.19947
H     0.41922   -0.75517    1.18708
H    -0.41922    0.75517   -1.18708
H     0.83934   -0.10941   -1.19947
H    -0.53746   -0.76618   -1.13106


## 8. Visualize symmetrized molecule

In [74]:
ase_mol_sym = Atoms([Atom(mol['atoms'][i], tuple(mol['coord'][i])) for i in range(len(mol['atoms']))])
mol_view_sym = ngl.show_ase(ase_mol_sym)
display(mol_view_sym)

NGLWidget()